In [1]:
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("WDC-DataFrame").getOrCreate()
#créer une session dans le master
spark = SparkSession.builder.master("spark://172.20.53.96:7077").appName("WDC-use-dico").getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-use-dico").getOrCreate()


#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/27 19:24:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/27 19:24:11 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
from pyspark.sql import functions as f

from pyspark.sql import Row
import re
from urllib.parse import urlparse

df = spark.read.text("s3a://test/")

#Split the line into four parts using the ' ' separator
split_col = f.split(f.col("value"), "\s+", 3)

#Create new columns for the subject, predicate, object, and graph
df = df.withColumn("subject", split_col[0])
df = df.withColumn("predicate", split_col[1])
df = df.drop("value")

# permet de ne plus différencier les predicats http, https, www...
df = df.withColumn("predicate", f.regexp_replace(f.col("predicate"), "([Hh][Tt][Tt][Pp][Ss]?://)?([Ww]{3}\.)?", ""))
df.show(truncate=0)

#df.createOrReplaceTempView("Super")
#spark.sql("select count(*) as count from Super").show()
 

23/03/27 19:24:13 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+-------------------------------------------------------------------------------------+--------------------------------------+
|subject                                                                              |predicate                             |
+-------------------------------------------------------------------------------------+--------------------------------------+
|<http://148.163.67.237/#website>                                                     |<w3.org/1999/02/22-rdf-syntax-ns#type>|
|<http://148.163.67.237/#website>                                                     |<schema.org/description>              |
|<http://148.163.67.237/#website>                                                     |<schema.org/inLanguage>               |
|<http://148.163.67.237/#website>                                                     |<schema.org/name>                     |
|<http://148.163.67.237/#website>                                                     |<schema.org/potentialAct

In [3]:
# Charge le dictionnaire
dico = spark.read.option("header",True) \
  .csv("s3a://test-out/dico_reel_total")
dico.createOrReplaceTempView("Dico")
spark.sql("select count(*) from Dico").show()
#dico.rdd.getNumPartitions()

+--------+
|count(1)|
+--------+
|   27865|
+--------+



In [4]:
# Attribut les id du dictionnaire au df
from pyspark.sql.functions import col

# Join df et dico sur la colomne "predicate"
joined_df = df.join(dico, on="predicate", how="left")

# Remplace la colomne "id" par "predicate"
final_df = joined_df.withColumn("predicate", col("id")).drop("id")
final_df.createOrReplaceTempView("Super")
final_df.show()

+---------+--------------------+
|predicate|             subject|
+---------+--------------------+
|        0|<http://148.163.6...|
|        7|<http://148.163.6...|
|        8|<http://148.163.6...|
|        1|<http://148.163.6...|
|       10|<http://148.163.6...|
|        2|<http://148.163.6...|
|        0|_:ne22bb1dfb7b445...|
|       22|_:ne22bb1dfb7b445...|
|       11|_:ne22bb1dfb7b445...|
|        0|<http://148.163.6...|
|        1|<http://148.163.6...|
|        2|<http://148.163.6...|
|        0|<http://148.163.6...|
|       24|<http://148.163.6...|
|       13|<http://148.163.6...|
|        8|<http://148.163.6...|
|        2|<http://148.163.6...|
|       12|<http://148.163.6...|
|        0|<http://148.163.6...|
|       16|<http://148.163.6...|
+---------+--------------------+
only showing top 20 rows



In [ ]:
%%time
## Mesure le temps d'exécution du characteristic set

# Vieille implem par Set, pas stockable en csv
#result=final_df.groupby("subject").agg(f.sort_array(f.collect_set(final_df.predicate)).alias("pset"))
#result2=result.groupby("pset").agg(f.approx_count_distinct(result.subject).alias('count')).sort(f.desc("count"))
#result2.show(truncate=0)

cset = spark.sql(f"select subject, concat_ws('|',sort_array(collect_set(predicate))) as pset FROM Super group by  subject ").cache()

result = cset.groupby("pset").agg(f.count(cset.subject).alias('count')).sort(f.desc('count'))
#result2=result.groupby("pset").agg(f.approx_count_distinct(result.subject).alias('count')).sort(f.desc("count"))
result.show(truncate=0)